In [ ]:
# CAN_direct.ipynb -- template for ananlyzing direct wave from Sumatra earthquake
# Carl Tape, Yongki Aiman*, Tomy Gunawan*, Angel Ling*, Amanda McPherson*
# the example waveform is from CAN (Canberra, Australia) for the 2004 Mw 9.X Sumatra-Andaman earthquake
# custom functions: markp_minutes, wf_fft

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pylab as plt
import warnings

from matplotlib import dates
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

from lib_seis import markp_minutes, wf_fft

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['lines.linewidth'] = 1

In [ ]:
# getting the waveform

client = Client("IRIS")

# 2004 Sumatra-Andaman earthquake
origintime = UTCDateTime("2004-12-26T00:58:53")
starttime  = origintime - (     12 * 60 * 60 )           # half a day before origin time
endtime    = origintime + ( 9 * 24 * 60 * 60)            # 9 days after origin time

st = client.get_waveforms("G", "CAN", "*", "LHZ", starttime, endtime, attach_response=True)
print(st)

In [ ]:
# plotting the seismogram

can = st.copy()                           

t   = can[0].times()                                           # time axis for plotting in seconds
tr  = can[0].data                                              

title = f'{can[0].stats["station"]} ({can[0].stats["channel"]}) − starting {can[0].stats["starttime"]}'

# plot the seismogram with time in seconds
plt.figure()
plt.plot(t, tr, 'b') 
plt.title(title)
plt.xlabel('time [s]')
plt.ylabel('counts');

In [ ]:
# zoom into the plot, and then pick an absolute time for the earthquake

st1 = st.copy()

t1 = starttime + (  9 * 60 * 60)                          # choose a start time for your window
t2 = starttime + ( 17 * 60 * 60)                          # choose an end time for your window

st1.trim(t1,t2)

t  = st1[0].times()
tr = st1[0].data

title = f'{st1[0].stats["station"]} ({st1[0].stats["channel"]}) − starting {st1[0].stats["starttime"]}'

plt.figure()
plt.plot(t, tr, 'b') 
plt.title(title)
plt.xlabel('time [s]')
plt.ylabel('counts');

In [ ]:
# spectral analysis on the extracted signal

st1.detrend('constant')
st1.detrend('linear')
st1.taper(max_percentage=0.2, type="cosine")

tr = st1[0].data

sampling_frequency = st1[0].stats.sampling_rate
Nyquist_frequency  = sampling_frequency / 2

fft_amp, fft_phase, fft_freq = wf_fft(tr, Nyquist_frequency)

C = fft_amp * np.exp( 1j * fft_phase )                     # Fourier transform reconstructed from its amplitude and phase